In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import dask
import dask.dataframe as dd

# Evitar que Dask convierta columnas object a string automáticamente
dask.config.set({"dataframe.convert-string": False})

# Ruta del dataset
dataset_path = "/kaggle/input/smadex-challenge-predict-the-revenue/train/train" 
# Para test, cambiar la ruta al archivo de test correspondiente

# Leer parquet con filtros aplicados
ddf = dd.read_parquet(dataset_path)

# Mostrar las primeras filas
print(ddf.head())



   buyer_d1  buyer_d7  buyer_d14  buyer_d28  buy_d7  buy_d14  buy_d28  \
0         0         1          1          1       1        1        1   
1         0         0          0          0       0        0        0   
2         0         0          0          0       0        0        0   
3         0         0          0          0       0        0        0   
4         0         0          0          0       0        0        0   

   iap_revenue_d7  iap_revenue_d14  iap_revenue_d28  ...  \
0        2.147718         2.147718         2.147718  ...   
1        0.000000         0.000000         0.000000  ...   
2        0.000000         0.000000         0.000000  ...   
3        0.000000         0.000000         0.000000  ...   
4        0.000000         0.000000         0.000000  ...   

                                   user_bundles_l28d  weekend_ratio  \
0  [88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...       0.019802   
1                                               None      

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
ddf.columns

In [ ]:
ddf[['dev_language_hist', 'iap_revenue_d7']].head(50)  #relacionar model amb el revenue, veiem 50

#filtrar els models que s'han gastat algo (revenue>0)
filtered = ddf[ddf['iap_revenue_d7'] > 0]
filtered[['dev_language_hist', 'iap_revenue_d7']].head(50)

#endreçant per revenue (descendent)
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
sorted_filtered[['dev_language_hist', 'iap_revenue_d7']].head(100)

#endreçant per revenue (ascendent)
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=True)
sorted_filtered[['dev_language_hist', 'iap_revenue_d7']].head(100)


In [ ]:
# Primer volem guarantir que ens dongui informació sobre els que han gastat alguna cosa
filtered = ddf[ddf['iap_revenue_d7'] > 0]

# Explicans el que més gasta
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)

# Mostra només els 15 primers que més gasten
top50 = sorted_filtered[['dev_language_hist', 'iap_revenue_d7']].head(50)
print("Top 50 usuarios que más gastaron:")
print(top50)

In [ ]:
# Primer volem guarantir que ens dongui informació sobre els que han gastat alguna cosa
filtered = ddf[ddf['iap_revenue_d7'] > 0]

# Explicans el que menys gasta sense incloure els que no gasten
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=True)

# Mostra només els 50 primers que meys gasten
top50 = sorted_filtered[['dev_language_hist', 'iap_revenue_d7']].head(50)
print("Top 50 usuarios que menos gastaron:")
print(top50)

In [ ]:
# Primer volem garantir que ens dongui informació sobre els que han gastat alguna cosa
filtered = ddf[ddf['iap_revenue_d7'] > 0]

# Explicans el que més gastan
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)

# Mostra només els 50 primers que més gasten
top50 = sorted_filtered[['region', 'iap_revenue_d7']].head(50)
print("Top 50 regiones que más gastaron:")
print(top50)

In [ ]:
# Filtrar usuarios que gastaron algo
filtered = ddf[ddf['iap_revenue_d7'] > 0]

# Filtrar filas donde region NO sea None, vacía o 'None'/'null'
def region_valida(x):
    return x not in (None, '', 'None', 'null')

# Usamos map con meta para evitar el warning
filtered = filtered[filtered['region'].map(region_valida, meta=('region', 'bool'))]

# Ordenar de de menos gasto a más gasto
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=True)

# Tomar los 50 primeros
top50 = sorted_filtered[['region', 'iap_revenue_d7']].head(50)
print("Top 50 regiones que menos gastaron:")
print(top50)



In [ ]:
#DEV_MODEL
ddf[['dev_model', 'iap_revenue_d7']].head(50)  #relacionar model amb el revenue, veiem 50

#endreçant per revenue (ascendent)
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=True)
sorted_filtered[['dev_model', 'iap_revenue_d7']].head(100)


In [ ]:
#Carrier
ddf[['carrier', 'iap_revenue_d7']].head(50)  #relacionar model amb el revenue, veiem 50

#filtrar els models que s'han gastat algo (revenue>0)
filtered = ddf[ddf['iap_revenue_d7'] > 0]
filtered[['carrier', 'iap_revenue_d7']].head(50)

#endreçant per revenue (descendent)
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
sorted_filtered[['carrier', 'iap_revenue_d7']].head(100)

In [ ]:
#DEV_MODEL
ddf[['dev_model', 'iap_revenue_d7']].head(50)  #relacionar model amb el revenue, veiem 50

#filtrar els models que s'han gastat algo (revenue>0)
filtered = ddf[ddf['iap_revenue_d7'] > 0]
filtered[['dev_model', 'iap_revenue_d7']].head(50)

#endreçant per revenue (descendent)
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
sorted_filtered[['dev_model', 'iap_revenue_d7']].head(100)



In [ ]:
#hour més alta - gastar més - no molt útil així que no ho considerem
# ddf[["hour", 'iap_revenue_d7']].head(1)
#filter
# filtered = ddf[ddf['iap_revenue_d7'] > 0]
# filtered[['hour', 'iap_revenue_d7']].head(1)

#ordenar
#sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
#sorted_filtered[['hour', 'iap_revenue_d7']].head(60)


In [ ]:
ddf[["country", 'iap_revenue_d7']].head(1)
#filter
filtered = ddf[ddf['iap_revenue_d7'] > 0]
filtered[['country', 'iap_revenue_d7']].head(1)

#ordena i enseyam els primers
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
sorted_filtered[['country', 'iap_revenue_d7']].head(60)

#quants diners genera cada país, ordenat (aquí ja no tenim en compte el filtre)
# Guardem l'agrupació en una variable (no calcula res)
country_revenue = ddf.groupby('country')['iap_revenue_d7'].sum()

#endreçar ascendentment
country_revenue_sorted_asc = country_revenue.compute().sort_values(ascending=True)

#endreçar descendentment 
country_revenue_sorted_desc = country_revenue.compute().sort_values(ascending=False)
#veure els 15 primers de cada grup
country_revenue_sorted_desc.head(15)  # Els 15 països amb més ingressos
country_revenue_sorted_asc.head(15)   # Els 15 amb menys (o zero)


In [ ]:
ddf[["weekay", 'iap_revenue_d7']].head()
#filter
filtered = ddf[ddf['iap_revenue_d7'] > 0]
filtered[['weekday', 'iap_revenue_d7']].head(50)

#ordenar DESCENDENT i ensenyar els primers
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=False)
sorted_filtered[['country', 'iap_revenue_d7']].head(60)

#ordenar ASCENDENT i ensenyar els primers
sorted_filtered = filtered.sort_values('iap_revenue_d7', ascending=True)
sorted_filtered[['country', 'iap_revenue_d7']].head(60)

MIRAR QUANTS DINERS PER DIA:
#guardar l'agrupació, encara no calculem res
weekday_revenue = ddf.groupby('weekday')['iap_revenue_d7'].sum()
#endreçar descendentment
weekday_revenue_sorted_desc =weekday_revenue.compute().sort_values(ascending=False)
#veure els 7 dies endreçats
weekday_revenue_sorted_desc.head(7)  # Els 7 dies endreçats per més ingressos


In [ ]:
#TIPUS D'APPS:
df_sorted = ddf[['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'iap_revenue_d7']]\
    .sort_values('iap_revenue_d7', ascending=False)

df_sorted.head(50)

In [ ]:
#El mateix, invertit
df_sorted = ddf[['advertiser_bundle', 'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'iap_revenue_d7']]\
    .sort_values('iap_revenue_d7', ascending=True)

df_sorted.head(50) #veiem que aquests jocs no treuen gaire diners


In [ ]:
#guardar l'agrupació, encara no calculem res
app_revenue = ddf.groupby('advertiser_bundle')['iap_revenue_d7'].sum()
#endreçar descendentment
app_revenue_sorted_desc = app_revenue.compute().sort_values(ascending=False)
#veure els 50 primers
app_revenue_sorted_desc.head(30)  # Els 30 països amb més ingressos

#Els adds poden saber que és, nosaltres no tenim aquesta info

In [3]:
filtered_ddf = ddf[
    (ddf['carrier'].notnull()) & 
    (ddf['dev_language_hist'].notnull())
]

ddf[['country', 'dev_language_hist', 'carrier', 'iap_revenue_d7']].sort_values('iap_revenue_d7', ascending=False).head(50)


,country,dev_language_hist,carrier,iap_revenue_d7
7144,co,"[(es, 150)]",movistar colombia,861191.142068
103936,es,None,telefonica de espana,701651.088080
68004,co,"[(es, 4)]",telmex colombia,696622.140000
11996,es,"[(es, 4), (unknown, 3)]",telefonica de espana,494950.883141
71086,es,None,telefonica de espana,494270.028662
130276,co,"[(en, 61)]",etb,455486.250000
4863,co,"[(es, 6)]",tigo colombia,425629.630000
2876,es,"[(es, 52)]",olivenet network s.l.,408452.130000
12811,es,"[(es, 59)]",telna,388803.330000
112512,es,"[(es, 1357)]",vodafone spain,359862.276410


In [ ]:
# ============================================================
# PIPELINE COMPLETO PARA PREDICCIONES DE REVENUE INVENT CHAT LAST ONE - COM ENFOCAR-HO CANVIANT LES RUTES... PER TAL DE PODER MÍNIM TENIR UN RESULTAT 
# ============================================================

# 1️⃣ IMPORTAR LIBRERÍAS
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

# 2️⃣ SELECCIONAR LOS ARCHIVOS (CAMBIAR RUTAS SI ES NECESARIO)
# En Kaggle podéis arrastrar los archivos a la carpeta /kaggle/input/
train_file = "/kaggle/input/smadex-challenge-predict-the-revenue/train/train"  # Cambiar a la ruta de vuestro train
test_file  = "/kaggle/input/smadex-challenge-predict-the-revenue/train/train"   # Cambiar a la ruta de vuestro test

# 3️⃣ CARGAR LOS ARCHIVOS
train_df = pd.read_parquet(train_file)
test_df  = pd.read_parquet(test_file)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# 4️⃣ SELECCIONAR FEATURES SIMPLES Y OBJETIVO
features = [
    "device_os",
    "device_os_version",
    "hour",
    "region"
]

target = "iap_revenue_d7"

# Comprobar que existen en el dataset
print("Columnas disponibles en train:", train_df.columns.tolist())

# 5️⃣ LIMPIEZA BÁSICA
train_df = train_df.fillna(-1)
test_df  = test_df.fillna(-1)

# 6️⃣ CODIFICAR VARIABLES CATEGÓRICAS (LightGBM necesita números)
for col in ["device_os", "region"]:
    train_df[col] = train_df[col].astype("category").cat.codes
    test_df[col]  = test_df[col].astype("category").cat.codes

# 7️⃣ SEPARAR TRAIN / VALIDATION (80%-20%)
X = train_df[features]
y = train_df[target]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Tamaño Train:", X_train.shape)
print("Tamaño Validation:", X_val.shape)

# 8️⃣ ENTRENAR MODELO LIGHTGBM
model = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64
)

print("Entrenando modelo...")
model.fit(X_train, y_train)
print("Entrenamiento completado.")

# 9️⃣ EVALUAR EN VALIDATION (opcional)
pred_val = model.predict(X_val)
pred_val = np.maximum(pred_val, 0)  # no negativos

msle = mean_squared_log_error(y_val, pred_val)
print("MSLE en validación:", msle)

# 10️⃣ PREDICCIONES SOBRE TEST
X_test = test_df[features]
pred_test = model.predict(X_test)
pred_test = np.maximum(pred_test, 0)  # no negativos

# 11️⃣ CREAR submission.csv
submission = pd.DataFrame({
    "row_id": test_df["row_id"],  # columna única que identifica cada fila
    "iap_revenue_d7": pred_test
})

submission.to_csv("submission.csv", index=False)
print("Archivo submission.csv generado correctamente.")

#A NOSALTRES ENS SURT KERNEL E
